# SVM
SVM的优缺点：
> 优点：泛化错误率低，计算开销不大，结果易解释。
> 缺点：对参数调节和核函数的选择较为敏感，原始分类器不做修改仅适用于二分类问题。
> 适用数据类型：数值型和标称型。

SVM的一般流程：
> 1. 收集数据:可以使用任意方法。
> 2. 准备数据:需要数值型数据。
> 3. 分析数据:有助于可视化分隔超平面。
> 4. 训练算法:SVM的大部分时间都源自训练,该过程主要实现两个参数的调优。
> 5. 测试算法:十分简单的计算过程就可以实现。
> 6. 使用算法:几乎所有分类问题都可以使用SVM,值得一提的是,SVM本身是一个二类分类器,对多类问题应用SVM需要对代码做一些修改。

In [1]:
import numpy as np
import matplotlib.pyplot as plt

使用拉格朗日乘子法可以得到SVM最优化问题的对偶问题如下：
$$
\begin{align}
\max_{\alpha}&\sum_{i=1}^{m}\alpha_i-\frac12\sum_{i=1}^m\sum_{j=1}^m\alpha_i\alpha_jy_iy_j\boldsymbol{x}_i^T\boldsymbol{x}_j \tag{1} \\
s.t. &\sum_{i=1}^m\alpha_iy_i=0 \\
&\alpha_i\geq0, i=1,2,...,m.
\end{align}
$$
我们的目标就是求解（1）式在约束条件下的最优解。这是一个二次规划问题。<br>
SMO的基本思路是先固定$\alpha_i$之外的所有参数，然后求$\alpha_i$上的极值。由于存在约束${\sum}_{i=1}^m\alpha_iy_i=0$，若固定$\alpha_i$之外的其他变量，则$\alpha_i$可以由其他变量导出。于是，SMO每次选择两个变量$\alpha_i$和$\alpha_j$，并固定其他参数，这样在初始化后，SMO不断执行以下两个步骤直至收敛：
> 1. 选取一对需要更新的变量$\alpha_i$和$\alpha_j$；
> 2. 固定$\alpha_i$和$\alpha_j$以外的参数，求解（1）式获得更新后的$\alpha_i$和$\alpha_j$。

## 1. 使用简化版SMO处理小规模数据集

In [2]:
def loadDataSet(fileName):
    dataMat = []; labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append(float(lineArr[0]), float(lineArr[1]))
        labelMat.append(float(lineArr[2]))
    return dataMat, labelMat

def selectJrand(i, m):
    """随机选择一个整数
    
    Parameters
    ------------
    i : 第一个alpha的下标
    m : 所有alpha的数目
    
    Returns
    ------------
    j : 一个不为i的随机数，在0～m之间
    """
    j = i
    while (j == i):
        j = int(np.random.uniform(0, m))
    return j

def clipAlpha(aj, H, L):
    """调整aj的值，使L<=aj<=H"""
    
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj

smoSimple，简化版的SMO算法函数：

In [4]:
def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    """SMO算法简化版
    
    Parameters
    -----------
    dataMatIn : 输入数据集
    classLabel : 类别标签
    C : 松弛变量，允许少量数据点被误分类，控制最大化间隔。
    toler : 容错率
    maxIter : 退出最大的循环次数
    
    Returns
    -----------
    b : 模型的截距值
    alphas : 拉格朗日乘子
    """
    
    dataMatrix = np.mat(dataMatIn); labelMat = np.mat(classLabels).T
    # 参数初始化
    b = 0; m, n = dataMatrix.shape
    alphas = np.mat(np.zeros((m, 1)))
    iter = 0
    while (iter < maxIter):
        # alphaPairsChanged记录alpha是否已经进行优化
        alphaPairsChanged = 0
        for i in range(m):
            # 预测的输出，y = w^Tx[i] + b;其中w = Σ(1~n) a[n]*lable[n]*x[n]
            fXi = float(np.multiply(alphas, labelMat).T * (dataMatrix * dataMatrix[i, :])) + b
            # Ei为误差
            Ei = fXi - float(labelMat[i])
            
            # KKT条件
            # 0<=alphas[i]<=C，但由于0和C是边界值，我们无法进行优化，因为需要增加一个alphas和降低一个alphas。
            # 表示发生错误的概率：labelMat[i]*Ei 如果超出了 toler， 才需要优化。
            if ((labelMat[i] * Ei < -toler) and (alphas[i] < C)) or \
                ((labelMat[i] * Ei > toler) and (alphas[i] > 0)):
                # 满足条件，选取非i的点进行优化比较
                j = selectJrand(i, m)
                # 预测j的结果
                fXj = float(np.multiply(alphas, labelMat).T * \
                      (dataMatrix * dataMatrix[j, :].T)) + b
                Ej = fXj - float(labelMat[j])
                alphaIold = alphas[i].copy()
                alphaJold = alphas[j].copy()
                
                # L和H用于将alphas[j]调整到0～C之间，L == H则不作改变
                # labelMat[i] != labelMat[j]表示两个点在异侧，则相减，否则相加
                if (labelMat[i] != labelMat[j]):
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L == H: print("L == H"); continue
                    
                
                eta = 2.0 * dataMatrix[i, :] * dataMatrix[j, :].T - \
                      dataMatrix[i, :] * dataMatrix[i, :].T - \
                      dataMatrix[j, :] * dataMatrix[j, :].T
                
                if eta >= 0: print("eta >= 0"); continue
                alphas[j] -= labelMat[j] * (Ei - Ej) / eta
                alphas[j] = clipAlpha(alphas[j], H, L)
                if (abs(alphas[j] - alphaJold) < 0.00001):
                    print("j not moving enough"); continue
                alphas[i] += labelMat[j] * labelMat[i] * (alphaJold - alphas[j])
                b1 = b - Ei - labelMat[i] * (alphas[i] - alphaIold) * \
                     dataMatrix[i, :] * dataMatrix[i, :].T - \
                     labelMat[j] * (alphas[j] - alphaJold) * \
                     dataMatrix[i, :] * dataMatrix[j, :].T
                b2 = b - Ej - labelMat[i] * (alphas[i] - alphaIold) * \
                     dataMatrix[i, :] * dataMatrix[j, :].T - \
                     labelMat[j] * (alphas[j] - alphaJold) * \
                     dataMatrix[j, :] * dataMatrix[j, :].T
                if (0 < alphas[i]) and (C > alphas[i]): b = b1
                elif (0 < alphas[j]) and (C > alphas[j]): b = b2
                else: b = (b1 + b2) / 2.0
                alphaPairsChanged += 1
                print("iter: %d  i: %d, pairs changed %d" % (iter, i, alphaPairsChanged))
        if (alphaPairsChanged == 0): iter += 1
        else: iter = 0
        print("iteration number: %d" % iter)
    return b, alphas